# Analyse Covid-19 Fälle in Deutschland
Datenquelle: Bereinigte Daten der Johns Hopkins University

Pakete laden und Installieren im Terminal mit pip install und Paketenamen

In [116]:
import pandas as pd
import plotly.graph_objects as go
import math

Daten importieren

In [117]:
def plot_layout(fig, Titel): #funktion für die Plots
    fig.update_layout(titlefont ={'size':22}, #titelgröße
    title={'text': Titel,
    'y':0.95,
    'x':0.5,
    'xanchor':'center'}, #def. wo und wie der Titel aussehen und stehen soll
    hovermode = 'x', #def. der Mausoption wenn ich im graph mir die werte anzeigen lassen möchte
    autosize = False, #keine automatische größe
    width=1300, height=700, paper_bgcolor='LightSteelBlue') #größe des graph und farbe
    return fig

In [118]:
df = pd.read_csv('https://raw.githubusercontent.com/Lucas-Czarnecki/COVID-19-CLEANED-JHUCSSE/master/COVID-19_CLEAN/csse_covid_19_time_series_cleaned/time_series_covid19_cases_tidy.csv', parse_dates=['Date'])

In [119]:
df.head()

,Province_State,Country_Region,Latitude,Longitude,Date,Confirmed,Deaths,Recovered
0,Alberta,Canada,53.9333,-116.5765,2020-01-22,0,0,NaN
1,Alberta,Canada,53.9333,-116.5765,2020-01-23,0,0,NaN
2,Alberta,Canada,53.9333,-116.5765,2020-01-24,0,0,NaN
3,Alberta,Canada,53.9333,-116.5765,2020-01-25,0,0,NaN
4,Alberta,Canada,53.9333,-116.5765,2020-01-26,0,0,NaN


Mit df.head() lasse ich mir den Kopfbereich der Daten anzeigen und mit df['Date'].iloc[-1] lasse ich mir (-1) den letzten Eintrag ausgeben. Also Datum der letzten Datenaktualisierung

In [120]:
df['Date'].iloc[-1]

Timestamp('2020-09-06 00:00:00')

Umbenennung der Überschriften (mit implace=True bekomme ich keine wiedergabe der Tabelle. Lasse ich das weg dann schon. So wird jedoch das bestehende df abgeändert. Alternativ ich schreibe vor df.rename df= also df = df.rename...)

In [121]:
df.rename(columns={'Country_Region':'land', 'Confirmed':'infiziert', 'Deaths':'tote', 'Recovered':'gesundet', 'Date':'datum'}, inplace=True)

Filtern des df

In [122]:
df = df[df.land=='Germany']

In [123]:
# df.head() ersten 5 werte und mit df.tail bekomme ich die letzten 5 werte 
df.tail()

,Province_State,land,Latitude,Longitude,datum,infiziert,tote,gesundet
33200,NaN,Germany,51.165691,10.451526,2020-09-02,247411,9322,219904.0
33201,NaN,Germany,51.165691,10.451526,2020-09-03,248840,9322,221525.0
33202,NaN,Germany,51.165691,10.451526,2020-09-04,250283,9327,222802.0
33203,NaN,Germany,51.165691,10.451526,2020-09-05,251058,9329,223570.0
33204,NaN,Germany,51.165691,10.451526,2020-09-06,251728,9330,224215.0


In [124]:
df['sterberate'] = df['tote'] / df['infiziert']*100 #neue zeile Sterberate
df['aktive'] = df['infiziert'] - df['tote']  - df['gesundet']       #neue zeile aktive

In [125]:
df.tail()

,Province_State,land,Latitude,Longitude,datum,infiziert,tote,gesundet,sterberate,aktive
33200,NaN,Germany,51.165691,10.451526,2020-09-02,247411,9322,219904.0,3.767820,18185.0
33201,NaN,Germany,51.165691,10.451526,2020-09-03,248840,9322,221525.0,3.746182,17993.0
33202,NaN,Germany,51.165691,10.451526,2020-09-04,250283,9327,222802.0,3.726582,18154.0
33203,NaN,Germany,51.165691,10.451526,2020-09-05,251058,9329,223570.0,3.715874,18159.0
33204,NaN,Germany,51.165691,10.451526,2020-09-06,251728,9330,224215.0,3.706381,18183.0


Die Anzahl der Infizierten Personen und Deutschland ist abhämgig von der Anzahl der getesteten Personen in Deutschland. 

In [126]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['gesundet']+df['tote'], mode='lines+markers', name='Abgeschlossene'))
fig.add_trace(go.Scatter(x=df['datum'], y=df['infiziert'], mode='lines+markers', name='Aktive', fill='tonexty'))
fig.add_trace(go.Scatter(x=df['datum'], y=df['infiziert'], mode='lines+markers', name='Infizierte'))
fig = plot_layout(fig, 'Summe der Infizierten in Deutschland')
fig.show()

In [127]:
df['neu_infiziert'] = df['infiziert'].diff()
fig = go.Figure()
fig.add_trace(go.Bar(x=df['datum'], y=df['neu_infiziert']))
fig = plot_layout(fig, 'Tägliche Neuinfektionen')
fig.show()

In [128]:
df['proz_steig_infiziert'] = df['neu_infiziert']/df['infiziert']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['proz_steig_infiziert'], mode='lines+markers'))
fig = plot_layout(fig, 'Steigungsrate Infizierte in %')
fig.show()

In [129]:
p = df['proz_steig_infiziert'].iloc[-1]
q = 1 + p/100
t = math.log(2)/math.log(q)
print(f'Verdopplung der infizierten Personen alle {t:.2f} Tage')

Verdopplung der infizierten Personen alle 260.77 Tage


In [130]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['aktive'], mode='lines+markers'))
fig = plot_layout(fig, 'Summe der Aktiven Personen in Deutschland')
fig.show()

In [131]:
df['neu_aktive'] = df['aktive'].diff()
fig = go.Figure()
fig.add_trace(go.Bar(x=df['datum'], y=df['neu_aktive']))
fig = plot_layout(fig, 'Tägliche neuen aktiven Neuinfektionen')
fig.show()

In [132]:
df['proz_steig_aktive'] = df['neu_aktive']/df['aktive']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['proz_steig_aktive'], mode='lines+markers'))
fig = plot_layout(fig, 'Steigungsrate Aktive Infizierte in %')
fig.show()

In [133]:
p = df['proz_steig_aktive'].iloc[-1]
q = 1 + p/100
t = math.log(2)/math.log(q)
print(f'Verdopplung der aktiven Personen alle {t:.2f} Tage')

Verdopplung der aktiven Personen alle 525.49 Tage


In [134]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['tote'], mode='lines+markers'))
fig = plot_layout(fig, 'Summe der Verstorbenen Personen in Deutschland')
fig.show()

In [135]:
df['neu_tote'] = df['tote'].diff()
fig = go.Figure()
fig.add_trace(go.Bar(x=df['datum'], y=df['neu_tote']))
fig = plot_layout(fig, 'Tägliche Verstorben')
fig.show()

In [136]:
df['proz_steig_tote'] = df['neu_tote']/df['tote']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['datum'], y=df['proz_steig_tote'], mode='lines+markers'))
fig = plot_layout(fig, 'Steigungsrate Stärbefälle Infizierte in %')
fig.show()

In [137]:
print(f'Sterberate in Deutschland aktuell bei {df["sterberate"].iloc[-1]:.2f} %')

Sterberate in Deutschland aktuell bei 3.71 %
